In [1]:
import sqlite3
import time
from datetime import datetime
import requests

In [3]:
def sendMessage(para, mensaje):
    """
    Envía un mensaje a través de la API de UTEA-CHAT con manejo de errores.
    
    - `para`: Número de teléfono (sin el código de país).
    - `mensaje`: Contenido del mensaje a enviar.
    """
    
    url = "http://localhost:8080/message/sendText/Jose-Dron"
    payload = {
        "number": '591' + para,  # Agrega el código de país (Bolivia: 591)
        "text": mensaje,
    }
    headers = {
        "apikey": "FCAC2E8AA2D8-4BC1-8A5F-34271779A659",
        "Content-Type": "application/json"
    }

    try:
        response = requests.request("POST", url, json=payload, headers=headers, timeout=10)

        # Verifica si la respuesta fue exitosa (código 200-299)
        response.raise_for_status()
        print(f"✅ Mensaje enviado correctamente a {para}: {mensaje}")
        return response.json()  # Retorna la respuesta en formato JSON

    except requests.ConnectionError:
        print(f"❌ Error de conexión al intentar enviar mensaje a {para}.")

    except requests.Timeout:
        print(f"⏳ Tiempo de espera agotado al intentar enviar mensaje a {para}.")

    except requests.HTTPError as e:
        print(f"⚠️ Error HTTP {response.status_code} al enviar mensaje a {para}: {response.text}")

    except requests.RequestException as e:
        print(f"❌ Error inesperado al enviar mensaje a {para}: {e}")

    return None  # Retorna None si ocurrió un error


In [4]:
def get_mensajes_sin_enviar():
    conn = sqlite3.connect('mensajes_whatsapp.db')
    cursor = conn.cursor()
    cursor.execute('''
        SELECT * FROM mensajes_whatsapp WHERE enviado = 0;
    ''')
    pendientes = cursor.fetchall()
    conn.close()
    return pendientes

In [5]:
def marcar_como_enviado(mensaje_id):
    intentos = 5  # cantidad máxima de intentos si la DB está bloqueada
    espera = 10  # segundos de espera entre intentos

    for intento in range(intentos):
        try:
            with sqlite3.connect('mensajes_whatsapp.db', timeout=10) as conn:
                cursor = conn.cursor()
                fecha_actual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                cursor.execute('''
                    UPDATE mensajes_whatsapp
                    SET enviado = 1, fecha_enviado = ?
                    WHERE id = ?
                ''', (fecha_actual, mensaje_id))
                conn.commit()

                print(f"✅ Mensaje con ID {mensaje_id} marcado como enviado.")
                break  # Sale del loop si se actualiza correctamente

        except sqlite3.OperationalError as e:
            print(f"⚠️ Intento {intento+1}/{intentos} - ID: {mensaje_id} - Base bloqueada: {e}")
            if intento < intentos - 1:  # Si no es el último intento, espera y reintenta
                time.sleep(espera)
            else:
                print(f"❌ Error: No se pudo actualizar después de {intentos} intentos. ID: {mensaje_id}")
    return None

In [6]:
def enviar_mensajes():
    mjs_pendientes = get_mensajes_sin_enviar()
    if len(mjs_pendientes) == 0:
        print(f"No existen mensajes nuevos para enviar...!!")
    for i in mjs_pendientes:
        idd = i[0]
        num = str(i[4])
        mjs = i[6]
        if (sendMessage(num, mjs)):
            marcar_como_enviado(idd)
            time.sleep(5)

In [ ]:
while True:
    enviar_mensajes()
    time.sleep(300)  # 600 segundos = 10 minutos

No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para enviar...!!
No existen mensajes nuevos para 

In [ ]:
msj_pendientes = get_mensajes_sin_enviar()
msj_pendientes

In [ ]:
msj_pendientes[1]

In [ ]:
msj_pendientes[1][6]

In [ ]:
conn = sqlite3.connect('mensajes_whatsapp.db')
cursor = conn.cursor()
# Borrar todos los mensajes de la tabla (pero no elimina la tabla)
cursor.execute('DELETE FROM mensajes_whatsapp;')
# Reiniciar el contador del ID autoincremental (opcional)
cursor.execute('DELETE FROM sqlite_sequence WHERE name="mensajes_whatsapp";')
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('mensajes_whatsapp.db')
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM mensajes_whatsapp WHERE enviado = 0;
''')
pendientes = cursor.fetchall()
for msg in pendientes:
    print(msg)
conn.close()